In [45]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [60]:
!brew install spatialindex



Running `brew update --auto-update`...
^C


In [ ]:
import glob
positions_list = []
measures_list = []
for csv in glob.glob("/Users/admin/Documents/Git_Repos/openaq-engine/local_data/plume_labs/*.csv"):
    if "positions" in str(csv):
        positions = pd.read_csv(csv)
        positions['date'] =  pd.to_datetime(positions['date'], format='%Y-%m-%d %H:%M:%S.%f')
        positions["minute_datetime"] = positions["date"].dt.round("min")
        positions_list.append(positions)
    if "measures" in str(csv):
        measures = pd.read_csv(csv)
        measures['date (UTC)'] = pd.to_datetime(measures['date (UTC)'], format='%Y-%m-%d %H:%M:%S.%f')
        measures["minute_datetime"] = measures["date (UTC)"].dt.round("min")
        measures_list.append(measures)
position_df = pd.concat(positions_list)
measures_df = pd.concat(measures_list)
measures_position = measures_df.merge(position_df, on='minute_datetime')
measures_position_gdf = gpd.GeoDataFrame(
    measures_position, geometry=gpd.points_from_xy(measures_position.longitude, measures_position.latitude)
)

measures_position_gdf.to_file("/Users/admin/Documents/Git_Repos/openaq-engine/local_data/plume_labs/locations_concentrations.geojson", driver="GeoJSON")

    

In [52]:
def pointInBBox(row, rect):
    x1, y1, x2, y2 = rect
    if (x1 < row["longitude"] and row["longitude"] < x2):
        if (y1 < row["latitude"] and row["latitude"] < y2):
            return True
    return False

In [ ]:
rect = [-71.124148, 42.343298,-71.067629, 42.368510]
boston_list = []
for csv in glob.glob("/Users/admin/Documents/Git_Repos/openaq-engine/local_data/predicted_data/*.csv"):
    subset_df = pd.DataFrame()
    df = pd.read_csv(csv)
    df["in_boston"] = df.apply(lambda row: pointInBBox(row, rect), axis=1)
    boston = df[(df['in_boston'] == True)]
    print(boston)
boston_list.append(boston)
boston_df = pd.concat(boston_list)


In [55]:
parcels = gpd.read_file("/Users/admin/Documents/Git_Repos/openaq-engine/local_data/geo_data/Cambridge/parcels.geojson", driver="GeoJSON")

In [62]:
parcels

,POLY_TYPE,MAP,UYEAR,LOC_ID,GlobalID,EditDate,ML,SOURCE,created_user,Editor,LOT,created_date,PLAN_ID,last_edited_date,last_edited_user,geometry
0,PRIV_ROW,-,0.0,F_761786_2962533,{11247B7E-4DC3-4EA4-9E8E-8C99160C2563},None,---,ASSESS,None,None,-,None,None,2022-10-05,SSWEENEY,"POLYGON ((-71.10911 42.37663, -71.10912 42.376..."
1,None,263,0.0,F_754404_2965391,{03FB4AC3-0F91-4074-AB4A-81A5921ADAF8},None,263-23,ASSESS,None,None,23,None,None,2022-10-05,SSWEENEY,"POLYGON ((-71.13621 42.38470, -71.13640 42.384..."
2,None,201,0.0,F_758048_2966643,{B330224E-22F2-4025-8EA5-4C3D608179DD},None,201-95,ASSESS,None,None,95,None,None,2022-10-05,SSWEENEY,"POLYGON ((-71.12275 42.38791, -71.12283 42.387..."
3,None,26,0.0,F_768259_2959811,{F30490E8-0660-4E22-95DE-AB4B2F6A75DB},None,26-37,ASSESS,None,None,37,None,None,2022-10-05,SSWEENEY,"POLYGON ((-71.08510 42.36914, -71.08511 42.369..."
4,None,36,0.0,F_767412_2961233,{5FC0A9B4-5EF3-4F0D-8F7B-4534CD39AD65},None,36-203,ASSESS,None,None,203,None,None,2022-10-05,SSWEENEY,"POLYGON ((-71.08823 42.37308, -71.08825 42.372..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12967,None,80,0.0,F_766216_2960859,{FE835112-18BC-4321-A0E4-51AB543B1F75},None,80-96,ASSESS,None,None,96,None,None,2022-10-05,SSWEENEY,"POLYGON ((-71.09263 42.37214, -71.09277 42.371..."
12968,None,201,0.0,F_757554_2966452,{1530D500-8343-4ACF-9973-CEBDECC2B996},None,201-24,ASSESS,None,None,24,None,None,2022-10-05,SSWEENEY,"POLYGON ((-71.12459 42.38754, -71.12454 42.387..."
12969,None,96,0.0,F_762631_2955736,{08BD170E-9FBD-4CE5-80FA-E3837133DBBA},None,96-102,ASSESS,None,None,102,None,None,2022-10-05,SSWEENEY,"POLYGON ((-71.10582 42.35803, -71.10605 42.357..."
12970,None,255,0.0,F_750255_2962034,{5FC33118-AFA9-4B3E-B60C-76A238981EBA},None,255-82,ASSESS,None,None,82,None,None,2022-10-05,SSWEENEY,"POLYGON ((-71.15150 42.37533, -71.15162 42.375..."


In [58]:
measures_position_gdf.sjoin(parcels)

/Users/admin/.pyenv/versions/3.10.4/lib/python3.10/site-packages/geopandas/geodataframe.py:2061: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


ImportError: Spatial indexes require either `rtree` or `pygeos`. See installation instructions at https://geopandas.org/install.html